# Investigating competition effects between 8 theoretical pre-miRNAs

## Set up workspace

In [1]:
import model_utils as mu
import params
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from tqdm import tqdm

In [2]:
destdir_csvs = './outputs/8sp-comp/csvs/'
destdir_figs = './outputs/8sp-cop/plots/'

## Generate model

In [3]:
#make 2 miRNA model
ODE_inits = [0]*8*4 + [0]
ODE_ka = [0] * 8
ODE_kb = ODE_ka
ODE_kc = ODE_ka
ODE_kd = ODE_ka

In [4]:
ode = mu.makeODEs(ODE_inits, ODE_ka, ODE_kb, ODE_kc, ODE_kd)

In [5]:
model = mu.makeModel(ode)

## Set up conditions for models

In [21]:
high = 10
low = 1
k = 2

In [22]:
inits1 = [params.init_pMiR1, params.init_pMiR1_dcr, params.init_pMiR1_dcr_star, params.init_MiR1] * 8

In [23]:
ka = [params.ka1 * high, params.ka1 * low] + [params.ka1] * 2 +\
     [params.ka1 * high * k, params.ka1 * low] + [params.ka1 * k] * 2

kb = [params.ka1 * params.Kd_wt, params.ka1 * params.Kd_wt * high]  + \
    [params.ka1 * params.Kd_wt] * 2 + \
    [params.ka1 * params.Kd_wt, params.ka1 * params.Kd_wt * high * k] +\
    [params.ka1 * params.Kd_wt] * 2

kc = [params.kc1] * 2 + [params.kc1 * high, params.kc1] + \
     [params.kc1] * 2 + [params.kc1 * high * k, params.kc1]

kd = [params.kd] * 3 + [params.kd * high] + [params.kd] * 4 + [params.kd * high * k]

In [24]:
dvars = np.linspace(0, 2.5, 5)
dvars[0] = 0.01
dvars = np.append(dvars, np.linspace(7.25, 17.25, 5))
variations = [str(int(i * 100)) for i in list(dvars)]

## Run model, save outputs

In [25]:
exec(model)

In [26]:
names = []
for i in tqdm(range(len(dvars))):
    init = inits1 + [dvars[i]]
    
    res = solve_ivp(runModel, (0, int(params.minutes)), init, \
                    args = (ka, kb, kc, kd), method = 'LSODA')
    
    mirna_length = int((len(res.y)-1)/4)
    index = []
    
    for j in range(mirna_length):
        index = index + ['pMiR' + str(j), 'pMiR_dicer' + str(j), \
                         'pMiR_dicer_star' + str(j), 'miR' + str(j)]
    
    index = index + ['dicer']
    
    df = pd.DataFrame(res.y, index = index)
    df = df.transpose()
    df.insert(loc = 0, column = 'time_min', value = res.t)
    
    name = 'model_' + variations[i]
    names.append(name)
    
    df.to_csv(destdir_csvs + name + '.csv', index = False)

100%|██████████| 10/10 [00:00<00:00, 39.35it/s]


## Plot models

### Load data

In [28]:
for name in names:
    exec(name  + ' = pd.read_csv("' + destdir_csvs + name + '.csv", header = [0])')

In [30]:
#mirna species names
mirnas = ['high $k_a$', 'high $k_b$', 'high $k_c', 'high_= $k_d$']
mirnas += [str(int(k)) + 'x ' + i for i in mirnas]

In [32]:
names[-1]

'model_1725'

In [33]:
#prep endpoints for plotting
ends = {'0': [], '1': [], '2': [], '3': [], '4': [], '5': [], '6': [], '7': []}
fc = ends

for i in range(len(dvars)):
    exec('miR = ' + names[i])
    
    ends['0'].append(miR['miR0'].iloc[-1])
    ends['1'].append(miR['miR1'].iloc[-1])
    ends['2'].append(miR['miR2'].iloc[-1])
    ends['3'].append(miR['miR3'].iloc[-1])
    ends['4'].append(miR['miR4'].iloc[-1])
    ends['5'].append(miR['miR5'].iloc[-1])
    ends['6'].append(miR['miR6'].iloc[-1])
    ends['7'].append(miR['miR7'].iloc[-1])
    
    fc['0'].append(miR['miR0'].iloc[-1] / model_1725['miR0'].iloc[-1])
    fc['1'].append(miR['miR1'].iloc[-1] / model_1725['miR1'].iloc[-1])
    fc['2'].append(miR['miR2'].iloc[-1] / model_1725['miR2'].iloc[-1])
    fc['3'].append(miR['miR3'].iloc[-1] / model_1725['miR3'].iloc[-1])
    fc['4'].append(miR['miR4'].iloc[-1] / model_1725['miR4'].iloc[-1])
    fc['5'].append(miR['miR5'].iloc[-1] / model_1725['miR5'].iloc[-1])
    fc['6'].append(miR['miR6'].iloc[-1] / model_1725['miR6'].iloc[-1])
    fc['7'].append(miR['miR7'].iloc[-1] / model_1725['miR7'].iloc[-1])